# Creating surface forcing

In [11]:
from roms_tools import Grid, SurfaceForcing

As always, the first step is to create our grid. Let's start with a coarse-resolution grid with a horizontal resolution of 2000km/100 = 20km.

In [12]:
grid = Grid(
    nx=520,  # number of grid points in x-direction
    ny=450,  # number of grid points in y-direction
    size_x=1560,  # domain size in x-direction (in km)
    size_y=1350,  # domain size in y-direction (in km)
    center_lon=-17.75,  # longitude of the center of the domain
    center_lat=30,  # latitude of the center of the domain
    rot=-15,  # rotation of the grid (in degrees)
    N=40,  # number of vertical layers
    verbose=True,
)

INFO - === Creating the horizontal grid ===
INFO - Total time: 0.198 seconds
INFO - ========================================================================================================
INFO - === Creating the mask ===
INFO - Total time: 0.556 seconds
INFO - ========================================================================================================
INFO - === Generating the topography using ETOPO5 data and hmin = 5.0 meters ===
INFO - Reading the topography data: 0.021 seconds
INFO - Regridding the topography: 0.017 seconds
INFO - Smoothing the topography globally: 0.051 seconds
INFO - Smoothing the topography locally: 5.221 seconds
INFO - Total time: 5.320 seconds
INFO - ========================================================================================================
INFO - === Preparing the vertical coordinate system using N = 40, theta_s = 5.0, theta_b = 2.0, hc = 300.0 ===
INFO - Total time: 0.003 seconds
INFO - ===============================================

Next, we specify the temporal range that we want to make the surface forcing for.

In [13]:
from datetime import datetime

In [14]:
start_time = datetime(2023, 1, 1)
end_time = datetime(2023, 2, 28)

`ROMS-Tools` can create two types of surface forcing:

* physical surface forcing like 10m wind, shortwave radiation, and air temperature at 2m
* biogeochemical (BGC) surface forcing like atmospheric pCO2

Unlike initial conditions data, ROMS can read multiple surface forcing files, so we create these two types separately in the following sections.

## Physical surface forcing

In this section, we use ERA5 data to construct the physical surface forcing. There are two ways to access the data:

- **Stream directly from the cloud** (no download required)
- **Use locally pre-downloaded files** by specifying the path

**Streaming** is convenient: there's no need to download data in advance.  
**Local files**, on the other hand, can reduce initialization time if you already have the data available.

Let's explore both options with these keyword arguments:

In [15]:
surface_forcing_kwargs = {
    "grid": grid,
    "start_time": start_time,
    "end_time": end_time,
    "type": "physics",
    "model_reference_date": datetime(2000, 1, 1), # this is the default
}

### Streaming Cloud-based ERA5 Data

Let's begin by exploring the streaming approach. This method uses [ARCO (Analysis-Ready, Cloud Optimized) ERA5 data](https://github.com/google-research/arco-era5/?tab=readme-ov-file#025-pressure-and-surface-level-data) from the public Google Cloud Storage bucket:
```bash
gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3
```
To activate the streaming approach in ROMS-Tools, simply omit the path in the `source` or explicitly provide the cloud storage URL:

* `source = {"name": "ERA5"}`
* `source = {"name": "ERA5", "path": "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"}`

<div class="alert alert-info">

Note

Streaming **requires** `use_dask = True` since the underlying data format is Zarr, which supports lazy loading and chunked access via Dask. [Here](https://roms-tools.readthedocs.io/en/latest/using_dask.html) you can learn more about using `Dask` with `ROMS-Tools`.

</div>

In [16]:
SurfaceForcing?

In [17]:
%%time

surface_forcing = SurfaceForcing(
    **surface_forcing_kwargs,
    source={"name": "ERA5", "path": 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'},
    use_dask=True,
)

OSError: [Errno -36] NetCDF: Invalid argument: 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'

Instantiating the surface forcing took over 1.5 minutes, even though no computations have been performed yet due to Dask’s lazy evaluation. **This delay results from streaming data from the cloud.** 

The surface forcing variables are held in an `xarray.Dataset` that is accessible via the `.ds` property. Indeed, all variables are Dask arrays representing lazy-loaded data.

In [18]:
surface_forcing.ds

NameError: name 'surface_forcing' is not defined

`ROMS-Tools` has found 505 time stamps within our specified time range. Let's double-check that `ROMS-Tools` has selected the correct times.

In [ ]:
surface_forcing.ds.time

<div class="alert alert-info">

Note

The `time` variable shows relative time, i.e., days since the model reference date (here set to January 1, 2000 by default). The `abs_time` coordinate shows the absolute time. The ERA5 data provided to `ROMS-Tools` has hourly frequency; this temporal frequency is inherited by `surface_forcing`.
    
</div>

To visualize any of the surface forcing fields, we can use the `.plot` method.

In [ ]:
surface_forcing.plot("uwnd", time=0)

In [ ]:
surface_forcing.plot("swrad", time=15)

### Using Local Pre-Downloaded ERA5 Data
Next, let's make the same surface forcing but using pre-downloaded ERA5 data sitting at the following location.

In [ ]:
path = "/home/ctroupin/Downloads/f5768806c2337cf1535efaeedb5be48f/data_stream-oper_stepType-accum.nc"

You can also download your own ERA5 data, which must span the desired ROMS domain and temporal range. For more details, please refer to [this page](https://roms-tools.readthedocs.io/en/latest/datasets.html).

In [ ]:
%%time

surface_forcing_from_local_data = SurfaceForcing(
    **surface_forcing_kwargs,
    source={"name": "ERA5", "path": path},
    use_dask=False,
)

Instantiating the surface forcing was much faster this time (just over 10 seconds) because it did not require streaming data.

## 

In [ ]:
surface_forcing_from_local_data.ds

### Shortwave radiation correction

Global data products like ERA5 are known to have biases in radiation, largely due to uncertainties in cloud-radiative feedbacks. To address these biases, `ROMS-Tools` includes a correction mechanism. When the `correct_radiation` parameter is set to `True` (which is the default), a multiplicative correction factor is applied to the ERA5 shortwave radiation data. These correction factors are pre-computed based on the difference between ERA5 climatology and the COREv2 climatology.

As a result, the `surface_forcing` dataset we generated above includes a shortwave radiation correction, since the default setting for `correct_radiation` is `True`. 
In the following, we create surface forcing where no correction is applied. While this approach is possible, it is **not recommended** due to the known biases in the ERA5 radiation data.

In [ ]:
%%time

uncorrected_surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": path},
    type="physics",
    correct_radiation=False,  # default is True
    use_dask=True,
)

In [ ]:
uncorrected_surface_forcing.ds

Here is a plot of the downward short-wave radiation, as before, but now in its uncorrected version.

In [ ]:
uncorrected_surface_forcing.plot("swrad", time=15)

Let's plot the differences in corrected and uncorrected short-wave radiation.

In [ ]:
from roms_tools.plot import plot

In [ ]:
plot(
    (surface_forcing.ds["swrad"] - uncorrected_surface_forcing.ds["swrad"]).isel(
        time=15
    ),
    grid_ds=grid.ds,
    cmap_name="Blues_r",
)

### Wind Correction Near the Coast

ERA5’s 1/4° resolution is too coarse to capture realistic coastal winds, particularly the wind drop-off near the coast. This drop-off is caused by coastal orography and differences between marine and terrestrial surface layers. It plays a crucial dynamical role in coastal upwelling systems, where the wind stress curl in the drop-off zone significantly influences Ekman pumping and biogeochemical reaction rates.

To address this limitation, `ROMS-Tools` can apply an exponential reduction in wind magnitude near the coastline. This correction uses an e-folding scale of 12.5 km, smoothly transitioning from full offshore wind strength to a 40% reduction at the shore. The behavior is controlled by the `wind_dropoff` parameter, which is set to `False` by default.

In [ ]:
surface_forcing_with_wind_drop_off = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": path},
    type="physics",
    wind_dropoff=True,
    use_dask=True,
)

In [ ]:
surface_forcing_with_wind_drop_off.ds

Here is a plot of the zonal wind, as before, but now with reduced amplitude near the coasts.

In [ ]:
surface_forcing_with_wind_drop_off.plot("uwnd", time=0)

Here is the difference between the uncorrected and corrected wind.

In [ ]:
plot(
    (surface_forcing.ds["uwnd"] - surface_forcing_with_wind_drop_off.ds["uwnd"]).isel(
        time=0
    ),
    grid_ds=grid.ds,
    cmap_name="RdBu_r",
)

The plot above highlights differences concentrated in near-coastal areas. Comparing the sign of the difference plot with the original wind field shows that positive wind values have been reduced and negative values enhanced, indicating a consistent reduction in wind amplitude near the coast.

### Fine vs. coarse grid

`ROMS-Tools` provides the option to interpolate surface forcing fields onto a "coarse grid," which has a resolution that is a factor of 2 coarser than the original "fine grid." Interpolating onto the coarse grid is useful when the ROMS grid resolution is at least twice as fine as that of the raw data product (e.g., ERA5). In such cases, interpolating onto the fine grid would not provide additional information compared to the coarse grid, making the coarse grid a more efficient choice and saving computational resources.

To control the choice between fine and coarse grids, `ROMS-Tools` provides the `coarse_grid_mode` argument, which accepts the following options:

- "auto" (default): Automatically selects the appropriate grid based on the comparison of the source and target spatial resolutions.
- "always": Always interpolate onto the coarse grid.
- "never": Always interpolate onto the fine grid, bypassing the coarse grid.

`ROMS-Tools` generates an informational message to indicate whether the fine or coarse grid is being used. 

The surface forcings we have generated so far have used `coarse_grid_mode = "auto"`, and the info messages indicated that the fine grid was selected. 
This decision was made because the ROMS grid resolution of approximately 20 km is similar to the ERA5 resolution of 1/4° (which at 33°N is about 23 km) - not fine enough to warrant using the coarse grid.

Now, let's try using a grid with a higher resolution of 2000km/200 = 10km.

In [ ]:
grid_10km = Grid(
    nx=200,
    ny=200,
    size_x=2000,
    size_y=2000,
    center_lon=-18,
    center_lat=33,
    rot=-20,
    topography_source={
        "name": "SRTM15",
        "path": "/global/cfs/projectdirs/m4746/Datasets/SRTM15/SRTM15_V2.6.nc",
    },
)

In [ ]:
grid_10km.ds

In [ ]:
%%time

surface_forcing_for_grid_10km = SurfaceForcing(
    grid=grid_10km,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": path},
    type="physics",
    coarse_grid_mode="auto",  # default is "auto"
    use_dask=True,
)

The info message confirms that `ROMS-Tools` has interpolated onto the "coarse grid," as the coarsened 10 km grid has a resolution of 20 km, which is still finer than the ERA5 resolution.

In [ ]:
surface_forcing_for_grid_10km.ds

As expected, the lengths of the spatial dimensions `eta_rho` and `xi_rho` in the surface forcing dataset are now 102—half of their original lengths of 202 in the fine grid, and equal to the lengths of `eta_coarse` and `xi_coarse` in the grid dataset.

<div class="alert alert-info">

Note

The dimension names in `surface_forcing_for_grid_10km.ds` are somewhat confusingly called `eta_rho`, `xi_rho` (rather than `eta_coarse`, `xi_coarse`). This is due to a particularity of ROMS - it expects the dimension names `eta_rho`, `xi_rho` even if the coarse grid is used.

</div>

We could also force interpolation onto the fine grid, though this would unnecessarily increase computational cost without adding any additional information.

In [ ]:
%%time

fine_surface_forcing_for_grid_10km = SurfaceForcing(
    grid=grid_10km,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": path},
    type="physics",
    coarse_grid_mode="never",
    use_dask=True,
)

In [ ]:
fine_surface_forcing_for_grid_10km.ds

The spatial dimensions `eta_rho` and `xi_rho` in the surface forcing dataset remain 202, matching their original lengths in the `grid_10km`.

## Biogeochemical (BGC) surface forcing

We now generate BGC surface forcing by interpolating biogeochemical variables from a dataset with 1° horizontal resolution. Two different approaches are used:  

1. **CESM BGC Monthly Fields**  
2. **A Unified BGC Climatology**, which integrates observationally and model based sources:  
   - Atmospheric pCO₂ from <cite data-cite="landschutzer_decadal_2016">(Landschuetzer et al., 2016)</cite>
   - Iron, dust, NOx and NHy decompositions from CESM model simulations

Both BGC datasets are available at the following locations: 

In [ ]:
cesm_bgc_path = "/global/cfs/projectdirs/m4746/Datasets/CESM_REGRIDDED/CESM-surface_lowres_regridded.nc"
unified_bgc_path = "/global/cfs/projectdirs/m4746/Datasets/UNIFIED/BGCdataset.nc"

In [ ]:
%%time

cesm_bgc_surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "CESM_REGRIDDED", "path": cesm_bgc_path},
    type="bgc",
    use_dask=True,
)

In [ ]:
cesm_bgc_surface_forcing.ds

In [ ]:
%%time

unified_bgc_surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "UNIFIED", "path": unified_bgc_path, "climatology": True},
    type="bgc",
    use_dask=True,
)

In [ ]:
unified_bgc_surface_forcing.ds

<div class="alert alert-info">

**Note**  

The number of time stamps in the generated datasets differs due to their source data:  

- **CESM BGC (Monthly Data)**:  
  `cesm_bgc_surface_forcing.ds` contains three time entries—one before `start_time`, one after `end_time`, and one in between. The entries (at or) before and (at or) after `start_time` ensure compatibility with ROMS. If the input data does not meet this requirement, `ROMS-Tools` issues a warning.  

- **Unified BGC Climatology**:  
  Since this dataset is climatological (`source["climatology"] = True`), it always contains twelve time entries, unaffected by `start_time` and `end_time`. All `*_time` coordinates include a `cycle_length` attribute (in days), instructing ROMS to repeat the data every 365.25 days. This makes `unified_bgc_boundary_forcing.ds` suitable for any simulation period, provided the reference date is January 1 of any year.  

</div>

We can plot the BGC surface forcing as we saw above.

In [ ]:
cesm_bgc_surface_forcing.plot("pco2_air", time=0)

In [ ]:
unified_bgc_surface_forcing.plot("pco2_air", time=0)

In [ ]:
cesm_bgc_surface_forcing.plot("iron", time=0)

In [ ]:
unified_bgc_surface_forcing.plot("iron", time=0)

In [ ]:
cesm_bgc_surface_forcing.plot("nox", time=0)

In [ ]:
unified_bgc_surface_forcing.plot("nox", time=0)

## Saving as NetCDF or YAML file
Once we have decided which of the surface forcing versions we actually want to use, we can save the dataset as a NetCDF file.

We need to specify the desired target path.

In [ ]:
filepath = "/pscratch/sd/n/nloose/forcing/my_bgc_surface_forcing.nc"

In [ ]:
%time cesm_bgc_surface_forcing.save(filepath)

Alternatively, we can tell `ROMS-Tools` to group the surface forcing into multiple files ordered by months.

In [ ]:
filepath = "/pscratch/sd/n/nloose/forcing/my_surface_forcing.nc"

In [ ]:
%time surface_forcing.save(filepath, group=True)

From the file paths printed to the screen, you will notice that the year and month information has been appended to the specified path.

<div class="alert alert-info">

Note

The way data is grouped depends on its temporal frequency. For datasets with a daily or higher frequency, the data is grouped by month. For datasets with lower temporal frequency (e.g., monthly data), the grouping is done by year.

</div>

We can also export the parameters of our `SurfaceForcing` object to a YAML file.

In [ ]:
yaml_filepath = "/pscratch/sd/n/nloose/forcing/my_surface_forcing.yaml"

In [ ]:
surface_forcing.to_yaml(yaml_filepath)

This is the YAML file that was created.

In [ ]:
# Open and read the YAML file
with open(yaml_filepath, "r") as file:
    file_contents = file.read()

# Print the contents
print(file_contents)

## Creating surface forcing from an existing YAML file

In [ ]:
%time the_same_surface_forcing = SurfaceForcing.from_yaml(yaml_filepath, use_dask=True)

In [ ]:
the_same_surface_forcing.ds